In [1]:
from skfuzzy import control as ctrl
import pickle
import pandas as pd
import random

In [2]:
def categorize_outcome(outcome):
    if outcome < 40:
        return 'Loss'
    elif 40 <= outcome < 70:
        return 'Draw'
    else:
        return 'Win'

In [3]:
with open("match_outcome.pickle", "rb") as f:
   match_outcome_ctrl = pickle.load(f)

In [4]:
def run_fuzzy_system(team_performance, players_statistics, weather_conditions):
    match_outcome = ctrl.ControlSystemSimulation(match_outcome_ctrl)
    match_outcome.input["team_performance"] = team_performance
    match_outcome.input["players_statistics"] = players_statistics
    match_outcome.input["weather_conditions"] = weather_conditions
    match_outcome.compute()
    crisp_output = match_outcome.output["match_outcome"]

    match_health = categorize_outcome(crisp_output)

    return match_health

In [5]:
df = pd.read_excel("test_data.xlsx", engine='openpyxl')


In [6]:
df

,team_performance,players_statistics,weather_conditions,expected
0,poor,low,bad,Loss
1,excellent,high,good,Win
2,average,medium,moderate,Draw
3,poor,medium,good,Win
4,excellent,low,bad,Win
5,average,low,good,Win
6,poor,low,moderate,Draw
7,excellent,high,good,Win
8,average,medium,bad,Loss
9,poor,low,good,Draw


In [7]:
df["team_performance"] = df["team_performance"].map(
        {
            "poor": random.uniform(0, 10),
            "average": random.uniform(40, 60),
            "excellent": random.uniform(90, 100),
        }
    )

df["players_statistics"] = df["players_statistics"].map(
    {
        "low": random.uniform(0, 10),
        "medium": random.uniform(40, 60),
        "high": random.uniform(90, 100),
    }
)

df["weather_conditions"] = df["weather_conditions"].map(
    {
        "bad": random.uniform(0, 10),
        "moderate": random.uniform(40, 60),
        "good": random.uniform(90, 100),
    }
)

In [8]:
df.head()

,team_performance,players_statistics,weather_conditions,expected
0,0.545809,6.719070,7.694306,Loss
1,91.322504,94.711450,98.466892,Win
2,42.929916,51.503672,59.446371,Draw
3,0.545809,51.503672,98.466892,Win
4,91.322504,6.719070,7.694306,Win


In [9]:
def predict(row):
    return run_fuzzy_system(row["team_performance"], row["players_statistics"], row["weather_conditions"])

df["Predicted match outcome Condition"] = df.apply(predict, axis=1)

In [10]:
df

,team_performance,players_statistics,weather_conditions,expected,Predicted match outcome Condition
0,0.545809,6.719070,7.694306,Loss,Loss
1,91.322504,94.711450,98.466892,Win,Win
2,42.929916,51.503672,59.446371,Draw,Draw
3,0.545809,51.503672,98.466892,Win,Draw
4,91.322504,6.719070,7.694306,Win,Win
5,42.929916,6.719070,98.466892,Win,Draw
6,0.545809,6.719070,59.446371,Draw,Draw
7,91.322504,94.711450,98.466892,Win,Win
8,42.929916,51.503672,7.694306,Loss,Draw
9,0.545809,6.719070,98.466892,Draw,Draw


In [11]:
correct_predictions = (
    df["Predicted match outcome Condition"]
    == df["expected"]
).sum()

accuracy = correct_predictions / len(df)

print(f"Accuracy: {accuracy}")

Accuracy: 0.7


In [12]:
TP = ((df["Predicted match outcome Condition"] == "Win") & (df["expected"] == "Win")).sum()
FP = ((df["Predicted match outcome Condition"] == "Win") & (df["expected"] != "Win")).sum()
FN = ((df["Predicted match outcome Condition"] != "Win") & (df["expected"] == "Win")).sum()

# Calculate the precision for the "Win" class
precision = TP / (TP + FP)

print(f"Precision (Win class): {precision}")

Precision (Win class): 1.0
